In [11]:
import numpy as np
import pandas as pd

In [12]:
df = pd.read_csv('data/en_review/combined_output_only_review.csv')
df = df['review']
df

0      It is the starting point on the side of the To...
1      The station facility seems to be old, but not ...
2      This is a general train station. Keio's Asakus...
3      The station is crowded with office workers dur...
4      It is easy to find the way because it is well ...
                             ...                        
364    I went to Tokyo Metro Toy Zay and Minami Shoto...
365    There are all good things to live with reagent...
366    Ceramic Line Station.It is positive to say tha...
367    Tokyo Metro all over the world!Stamp Rally is ...
368    Despite being a pretty large transit station, ...
Name: review, Length: 369, dtype: object

In [13]:
reviews = []
for i in df:
  i = i.replace('\\n','\n').replace('\ n','\ N')
  j = i.split('\ N')
  for review in j:
    if review != ' ':
      reviews.append(review)
print(len(reviews))

9226


In [83]:
from skllm.config import SKLLMConfig

SKLLMConfig.set_openai_key('---') # API key in case you use it
SKLLMConfig.set_openai_org('---') # API organization in case you use it

In [85]:
# change when you use it
# created to split up the workload
start = 0
end = 9226
steps = 10

In [88]:
from skllm.models.gpt.classification.zero_shot import ZeroShotGPTClassifier


def sentiment_classifier_result(start,end,steps):

    result = pd.DataFrame(columns=['review', 'label'])
    
    clf = ZeroShotGPTClassifier()

    clf.fit(None,['Positive','Neutral','Negative'])

    for i in range(start, end, steps):
        tmp = reviews[i:i+steps]
        print(tmp)
        label = clf.predict(tmp)
        for j in range(len(label)):
            new_data = {'review': tmp[j], 'label': label[j]}
            result.loc[len(result)] = new_data
            result = result.reset_index(drop=True)
            
    return result
        

In [ ]:
sentiment_result = sentiment_classifier_result(start,end,steps)
sentiment_result

In [ ]:
positive_reviews = sentiment_result[sentiment_result['label']=='Positive'].drop(columns=['label'])
positive_reviews.to_csv("data/reviews_llm/postiive_reviews_llm.csv")
negative_reviews = sentiment_result[sentiment_result['label']=='Negative'].drop(columns=['label'])
negative_reviews.to_csv("data/reviews_llm/negative_reviews_llm.csv")